<a href="https://colab.research.google.com/github/trujoan/identificar-img/blob/main/Contador_de_dedor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Identificar y contar el número de dedos en una imagen de una mano humana.

In [ ]:
!pip install mediapipe opencv-python

In [ ]:
# Instalar las librerías en Google Colab

import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow
from google.colab import files

# Inicializar el módulo de manos de MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def procesar_imagenes():
    """
    Permite cargar entre 1 y 10 imágenes, y aplica el modelo de MediaPipe para detectar la mano y los dedos.
    Maneja dos contadores, uno para la mano derecha y otro para la izquierda.
    """
    uploaded = files.upload()  # Cargar las imágenes (deben ser entre 1 y 10)
    image_paths = list(uploaded.keys())

    if len(image_paths) < 1 or len(image_paths) > 10:
        print("Error: Debes cargar entre 1 y 10 imágenes.")
        return

    with mp_hands.Hands(static_image_mode=True, max_num_hands=2,
                        min_detection_confidence=0.7) as manos:
        for image_path in image_paths:
            # Leer la imagen
            image = cv2.imread(image_path)

            # Convertir la imagen de BGR a RGB para MediaPipe
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resultados = manos.process(image_rgb)

            # Inicializar contadores de dedos
            dedos_levantados_derecha = 0
            dedos_levantados_izquierda = 0

            # Si se detectan manos
            if resultados.multi_hand_landmarks and resultados.multi_handedness:
                for idx, hand_landmarks in enumerate(resultados.multi_hand_landmarks):
                    # Obtener si la mano es derecha o izquierda
                    mano_dominante = resultados.multi_handedness[idx].classification[0].label
                    if mano_dominante == "Right":
                        # Contar dedos levantados en la mano derecha
                        dedos_levantados_derecha = contar_dedos(hand_landmarks, True)
                    else:
                        # Contar dedos levantados en la mano izquierda
                        dedos_levantados_izquierda = contar_dedos(hand_landmarks, False)

                    # Dibujar los puntos y conexiones de la mano en la imagen
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Mostrar los contadores en la imagen
                cv2.putText(image, f'Mano derecha: {dedos_levantados_derecha}', (50, 25),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0, 255, 0), 1, cv2.LINE_AA)
                cv2.putText(image, f'Mano izquierda: {dedos_levantados_izquierda}', (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (255, 0, 0), 1, cv2.LINE_AA)

            # Mostrar la imagen procesada
            cv2_imshow(image)

def contar_dedos(hand_landmarks, es_mano_derecha):
    """
    Cuenta cuántos dedos están levantados usando los landmarks de MediaPipe.
    Diferencia entre mano derecha e izquierda para ajustar el conteo del pulgar.

    Parámetros:
    hand_landmarks: Lista de landmarks detectados en la mano.
    es_mano_derecha: Bool, True si es la mano derecha, False si es la izquierda.

    Retorna:
    int -> Número de dedos levantados.
    """
    dedos_levantados = 0

    # Obtener los puntos clave de la mano (landmarks)
    puntos = hand_landmarks.landmark

    # Para el pulgar, la lógica cambia dependiendo de si es la mano derecha o izquierda
    if es_mano_derecha:
        if puntos[mp_hands.HandLandmark.THUMB_TIP].x < puntos[mp_hands.HandLandmark.THUMB_IP].x:
            dedos_levantados += 1
    else:  # Para la mano izquierda
        if puntos[mp_hands.HandLandmark.THUMB_TIP].x > puntos[mp_hands.HandLandmark.THUMB_IP].x:
            dedos_levantados += 1

    # Dedos índice, medio, anular y meñique (comparar la y-position de la punta con la base)
    dedos = [mp_hands.HandLandmark.INDEX_FINGER_TIP,
             mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
             mp_hands.HandLandmark.RING_FINGER_TIP,
             mp_hands.HandLandmark.PINKY_TIP]

    bases = [mp_hands.HandLandmark.INDEX_FINGER_MCP,
             mp_hands.HandLandmark.MIDDLE_FINGER_MCP,
             mp_hands.HandLandmark.RING_FINGER_MCP,
             mp_hands.HandLandmark.PINKY_MCP]

    for dedo, base in zip(dedos, bases):
        if puntos[dedo].y < puntos[base].y:
            dedos_levantados += 1

    return dedos_levantados


# Llamar a la función para procesar imágenes
procesar_imagenes()